In [27]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import nltk 
import re
import warnings
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

data = pd.read_csv('comment.csv')
data.head()

,Page Number,Comment Number,Comment Text,Star Count,Seller Info,Any Info
0,1,1,Cok kisa surede cok saglam geldi..urunun gors...,5,bu ürünü Hızlıgitti satıcısından aldı.,NaN
1,1,2,"Kullanımı kolay , zaman ayarlamasını yapıyorsu...",5,bu ürünü EMC&BİLİSİM&TEKN satıcısından aldı.,NaN
2,1,3,4 kişilik cekırdek bır aileye yeterli bence.4 ...,5,bu ürünü Hepsiburada satıcısından aldı.,NaN
3,1,4,Ürünü çok beğendim daha önce almadığım için pi...,5,bu ürünü Hızlıgitti satıcısından aldı.,NaN
4,1,5,Kanat yaptım çok güzeldi çok beğendim mükemmel...,5,bu ürünü Hızlıgitti satıcısından aldı.,NaN


In [28]:
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65 entries, 0 to 64
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Page Number     65 non-null     object
 1   Comment Number  65 non-null     object
 2   Comment Text    65 non-null     object
 3   Star Count      65 non-null     object
 4   Seller Info     65 non-null     object
 5   Any Info        5 non-null      object
dtypes: object(6)
memory usage: 3.2+ KB


In [29]:
WPT = nltk.WordPunctTokenizer()
stop_word_list = nltk.corpus.stopwords.words("turkish")


docs = data["Comment Text"]
docs = docs.map(lambda x: re.sub('[,\.!?();:$%&#"]', '', x))
docs = docs.map(lambda x: x.lower())
docs = docs.map(lambda x: x.strip())

def token(values):
    filtered_words =  [word for word in values.split() if word not in stop_word_list]
    not_stopword_doc = " ".join(filtered_words)
    return not_stopword_doc

docs = docs.map(lambda x: token(x))
data["Comment Text"] = docs

print(data.head(20))

    Page Number  Comment Number  \
0             1               1   
1             1               2   
2             1               3   
3             1               4   
4             1               5   
5             1               6   
6             1               7   
7             1               8   
8             1               9   
9             1              10   
10  Page Number  Comment Number   
11            2               1   
12            2               2   
13            2               3   
14            2               4   
15            2               5   
16            2               6   
17            2               7   
18            2               8   
19            2               9   

                                         Comment Text  Star Count  \
0   cok kisa surede cok saglam geldiurunun gorsell...           5   
1   kullanımı kolay zaman ayarlamasını yapıyorsunu...           5   
2   4 kişilik cekırdek bır aileye yeterli bence4 a...    

In [30]:
dataDoc = data["Comment Text"].values.tolist()
dataStar = data["Star Count"].values.tolist()

X_train, X_test, y_train, y_test = train_test_split(dataDoc,dataStar, test_size=0.3 , random_state=42)

tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2),max_df=0.9, min_df=5)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)


In [31]:
words_counts = Counter([word for line in X_train for word in line.split(' ')])

most_common_words = sorted(words_counts.items(), key=lambda x : x[1], reverse=True)[:20]

most_common_words[:20]

[('güzel', 12),
 ('ürün', 11),
 ('bir', 9),
 ('tavuk', 6),
 ('oldu', 5),
 ('pratik', 5),
 ('kişilik', 5),
 ('gayet', 5),
 ('yeterli', 5),
 ('harika', 5),
 ('ürünü', 5),
 ('iyi', 5),
 ('comment', 5),
 ('text', 5),
 ('patates', 4),
 ('tek', 4),
 ('hızlı', 4),
 ('memnun', 4),
 ('ederim', 4),
 ('pat', 4)]

In [32]:
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.multiclass import OneVsRestClassifier

In [33]:
#lineersvc
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
SVC_pipeline = Pipeline([
                ('clf', OneVsRestClassifier(LinearSVC(), n_jobs=1)),
            ])
SVC_pipeline.fit(X_train_tfidf, y_train)
prediction = SVC_pipeline.predict(X_test_tfidf)
print('LineerSVC accuracy is {}'.format(accuracy_score(y_test, prediction)))



LineerSVC accuracy is 0.75


C:\Users\alper\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Users\alper\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Users\alper\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\User

In [34]:
#stochastic gradient descent
model2 = OneVsRestClassifier(SGDClassifier(loss = 'hinge', penalty = 'elasticnet', max_iter = 5))
model2.fit(X_train_tfidf, y_train)
print ("SGD Accuracy={}".format(accuracy_score(y_test, model2.predict(X_test_tfidf))))
sgdpred = accuracy_score(y_test, model2.predict(X_test_tfidf))

SGD Accuracy=0.75


C:\Users\alper\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_stochastic_gradient.py:713: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
C:\Users\alper\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_stochastic_gradient.py:713: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
C:\Users\alper\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_stochastic_gradient.py:713: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
C:\User

In [39]:
from sklearn.decomposition import LatentDirichletAllocation as LDA
from sklearn.feature_extraction.text import CountVectorizer

def train_lda_model(data, n_topics=30):
    # Count vectorization
    tf_vectorizer = CountVectorizer(max_df=0.95, min_df=1, max_features=10000)
    tf = tf_vectorizer.fit_transform(data)
    tf_feature_names = tf_vectorizer.get_feature_names_out()

    # LDA model
    lda = LDA(n_components=n_topics, max_iter=5, learning_method='online', learning_offset=50., random_state=0)
    lda.fit(tf)

    return lda, tf, tf_feature_names  # Include tf in the return to access it later

def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

# Example usage:
lda_model, tf_matrix, tf_feature_names = train_lda_model(dataDoc)
display_topics(lda_model, tf_feature_names, 30)


Topic 0:
güzel göre düşük bir markaya aile kişilik fiyat kapasitesi makinaya teslimat kötü süper günü aldım et kargo çalışmıyor düşeni çıkıp almadığım gönderirken guzel dakika bence4 20 bitti 10 muhteşem kişi
Topic 1:
kargo tavuk mutfağım alsam küçüktü yeterli hızlıydı ürüne gelir söz günde yaptık büyük patateskestane oldu gelmez olacakmış belli markası kişilik hızlı xxl diyecek gayet marka ederim pratik almamak tavsiye teslim
Topic 2:
gücü kanat cok çizik düğmeye çıktı oldugum yeni puşmemisti haliyle size taşıma ic aileler tariflere şimdi doğramak 12 bir kullandıktan kitapçık biber mutlaka hediyesi sonra sıcaklığı durumda kuvvetli gelmez hediye
Topic 3:
koydumhizli kapatıp nerdeyse makinada koymayin yaptı iki marka hacım kalın vs güzel günde patatesin çıkmıyor surede teslimat oldugum kullandim ardından sağlam urun ic text em sökülmüş epey anneme markası beyler
Topic 4:
ürün kaliteli yaptım aldım bekliyoruz serviste kanat iyi sonuç tavsiye almadım iki arızalandı philips bir patates şim

In [40]:
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda,tf, tf_vectorizer, mds = 'tsne' )

ModuleNotFoundError: No module named 'pyLDAvis'